In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
from lutils.stock import LTdxHq

In [3]:
ddf = pd.read_pickle('d:/ddf.pkl').dropna()

In [4]:
ddf

,open,close,high,low,volume,amount,tic,date,day
1103,95.00,82.92,95.00,80.00,552103872.0,4.796717e+10,688981,2020-07-16,3
1104,79.00,77.06,84.90,75.00,219471584.0,1.738815e+10,688981,2020-07-17,4
1105,77.19,79.17,80.51,70.02,228583344.0,1.700523e+10,688981,2020-07-20,0
1106,78.30,78.63,82.89,77.77,161873264.0,1.297766e+10,688981,2020-07-21,1
1107,77.80,79.57,81.78,77.20,133942792.0,1.068249e+10,688981,2020-07-22,2
...,...,...,...,...,...,...,...,...,...
1467,1.15,1.14,1.16,1.12,41564420.0,4.722449e+07,600145,2022-01-12,2
1468,1.14,1.13,1.15,1.12,26707056.0,3.029393e+07,600145,2022-01-13,3
1469,1.08,1.07,1.10,1.07,68835016.0,7.400834e+07,600145,2022-01-14,4
1470,1.02,1.02,1.02,1.02,5638200.0,5.750964e+06,600145,2022-01-17,0


In [5]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [6]:
dl = ddf.loc[-252:,:]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [7]:
# corr.style.background_gradient(cmap='coolwarm')

In [8]:
# corr.sum().sort_values()

In [9]:
stock_codes = list(corr.sum().sort_values().index.values[:6])

In [10]:
stock_codes

['000921', '600365', '000609', '601868', '002372', '001965']

In [11]:
# stock_codes = ['000921', '002032', '300406', '603789']
# stock_codes = ['000609', '000921', '001965', '002372']

In [12]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000921 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 600365 min: 2018-03-20 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 000609 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 601868 min: 2021-09-28 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 002372 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 001965 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------


In [13]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [14]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [15]:
df['daily_variance'] = (df.high-df.low) / df.close

In [16]:
dl = df.loc[-252:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,000609,000921,001965,002372
tic,,,,
000609,1.000000,-0.229306,-0.229676,-0.292968
000921,-0.229306,1.000000,-0.396903,0.106719
001965,-0.229676,-0.396903,1.000000,-0.769836
002372,-0.292968,0.106719,-0.769836,1.000000


In [17]:
train = data_split(df, '2019-01-01','2022-01-01') # 2021-07-01 2022-01-01

In [18]:
train.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,5.27,5.29,5.32,5.25,5759311.0,30399482.0,000609,2019-01-15,1,-0.082550,5.946131,4.840869,48.063900,-59.971539,0.911034,5.441000,5.467333,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.013233
0,7.34,7.55,7.55,7.22,20004778.0,161089488.0,000921,2019-01-15,1,0.171956,7.579382,5.955618,57.955100,171.516755,60.304291,6.799667,6.725167,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.043709
0,7.39,7.42,7.43,7.34,1422853.0,11541926.0,001965,2019-01-15,1,-0.008600,7.496838,7.139162,49.760481,27.822945,12.144795,7.363000,7.493500,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.012129
0,11.70,12.04,12.08,11.68,4112392.0,64759288.0,002372,2019-01-15,1,0.160269,12.085685,10.869315,56.642725,165.038307,13.980941,11.421000,11.001833,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.033223
1,5.28,5.30,5.59,5.16,13215401.0,70665424.0,000609,2019-01-16,2,-0.076507,5.937906,4.832094,48.186940,-33.431536,13.031968,5.425000,5.483167,"[[0.000959630829728534, 0.00031221297698058804...",tic 000609 000921 001965 00...,0.081132


In [19]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
#         self.reward = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
#             log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
#             new_portfolio_value = self.portfolio_value * (1 + log_portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [20]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f'Stock Dimension: {stock_dimension}, State Space: {state_space}')

Stock Dimension: 4, State Space: 4


In [21]:
# ['daily_variance', 'change', 'log_volume', 'close','day', 'macd', 'rsi_30', 'boll_ub', 'dx_30']
# ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'boll_ub', 'dx_30', 'close_30_sma', 'close_60_sma'] # cci_30
tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [22]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [23]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo", model_kwargs = PPO_PARAMS, tensorboard_log='D:/code/python/logs')

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


In [24]:
agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=1000000)

d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Logging to D:/code/python/logs\ppo_1
begin_total_asset:100000
end_total_asset:154398.87560139806
Sharpe:  0.6851663684815609
begin_total_asset:100000
end_total_asset:145915.03048088079
Sharpe:  0.6199735532756807
----------------------------------
| time/              |           |
|    fps             | 367       |
|    iterations      | 1         |
|    time_elapsed    | 5         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 165634.42 |
----------------------------------
begin_total_asset:100000
end_total_asset:202472.0945188156
Sharpe:  1.0182071371881092
begin_total_asset:100000
end_total_asset:171553.46141976325
Sharpe:  0.8250735086226509
begin_total_asset:100000
end_total_asset:173184.2382231426
Sharpe:  0.8447714393297109
-----------------------------------------
| time/                   |             |
|    fps                  | 419         |
|    iterations           | 2           |
|    time_elapsed         | 9           |


begin_total_asset:100000
end_total_asset:136876.7907475024
Sharpe:  0.5341634557033907
begin_total_asset:100000
end_total_asset:153356.7971899355
Sharpe:  0.6884131481028288
begin_total_asset:100000
end_total_asset:138103.33651064863
Sharpe:  0.543533894307107
------------------------------------------
| time/                   |              |
|    fps                  | 472          |
|    iterations           | 8            |
|    time_elapsed         | 34           |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 9.284122e-09 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.68        |
|    explained_variance   | -2.38e-07    |
|    learning_rate        | 0.0001       |
|    loss                 | 2.1e+12      |
|    n_updates            | 70           |
|    policy_gradient_loss | -2.26e-06    |
|    reward               | 124695.95    |
|    std

-------------------------------------------
| time/                   |               |
|    fps                  | 475           |
|    iterations           | 14            |
|    time_elapsed         | 60            |
|    total_timesteps      | 28672         |
| train/                  |               |
|    approx_kl            | 1.1554221e-08 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.68         |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.0001        |
|    loss                 | 2.2e+12       |
|    n_updates            | 130           |
|    policy_gradient_loss | -3.37e-06     |
|    reward               | 152552.33     |
|    std                  | 1             |
|    value_loss           | 4.18e+12      |
-------------------------------------------
begin_total_asset:100000
end_total_asset:176007.3386063175
Sharpe:  0.8518452022815037
begin_total_asset:100000
end_tota

begin_total_asset:100000
end_total_asset:168427.78421051367
Sharpe:  0.800539990319413
begin_total_asset:100000
end_total_asset:166768.73890896264
Sharpe:  0.7783185668641426
begin_total_asset:100000
end_total_asset:182084.1283297226
Sharpe:  0.8924107900814207
------------------------------------------
| time/                   |              |
|    fps                  | 478          |
|    iterations           | 21           |
|    time_elapsed         | 89           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 8.760253e-09 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.68        |
|    explained_variance   | 1.79e-07     |
|    learning_rate        | 0.0001       |
|    loss                 | 2.11e+12     |
|    n_updates            | 200          |
|    policy_gradient_loss | -1.45e-06    |
|    reward               | 115693.086   |
|    st

...

-------------------------------------------
| time/                   |               |
|    fps                  | 490           |
|    iterations           | 460           |
|    time_elapsed         | 1919          |
|    total_timesteps      | 942080        |
| train/                  |               |
|    approx_kl            | 1.0797521e-08 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.68         |
|    explained_variance   | 1.19e-07      |
|    learning_rate        | 0.0001        |
|    loss                 | 1.74e+12      |
|    n_updates            | 4590          |
|    policy_gradient_loss | -3.22e-06     |
|    reward               | 133276.94     |
|    std                  | 1             |
|    value_loss           | 3.34e+12      |
-------------------------------------------
begin_total_asset:100000
end_total_asset:147485.80143542794
Sharpe:  0.63600072627849
begin_total_asset:100000
end_total

begin_total_asset:100000
end_total_asset:180767.56248886397
Sharpe:  0.889598393701881
begin_total_asset:100000
end_total_asset:140643.25073663474
Sharpe:  0.5761956013356071
begin_total_asset:100000
end_total_asset:174911.19736074298
Sharpe:  0.8478652408014601
------------------------------------------
| time/                   |              |
|    fps                  | 490          |
|    iterations           | 467          |
|    time_elapsed         | 1948         |
|    total_timesteps      | 956416       |
| train/                  |              |
|    approx_kl            | 8.032657e-09 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.68        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0001       |
|    loss                 | 2.19e+12     |
|    n_updates            | 4660         |
|    policy_gradient_loss | -5.82e-07    |
|    reward               | 116251.07    |
|    s

------------------------------------------
| time/                   |              |
|    fps                  | 490          |
|    iterations           | 473          |
|    time_elapsed         | 1973         |
|    total_timesteps      | 968704       |
| train/                  |              |
|    approx_kl            | 6.693881e-09 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.68        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0001       |
|    loss                 | 2.29e+12     |
|    n_updates            | 4720         |
|    policy_gradient_loss | -3.57e-07    |
|    reward               | 125587.24    |
|    std                  | 1            |
|    value_loss           | 4.52e+12     |
------------------------------------------
begin_total_asset:100000
end_total_asset:188908.758946689
Sharpe:  0.9458465157662833
begin_total_asset:100000
end_total_asset:153039.844190

begin_total_asset:100000
end_total_asset:147683.19745685608
Sharpe:  0.633652089103628
begin_total_asset:100000
end_total_asset:171482.03491258057
Sharpe:  0.8254320738772173
begin_total_asset:100000
end_total_asset:166386.54775121994
Sharpe:  0.7830960586957251
--------------------------------------------
| time/                   |                |
|    fps                  | 490            |
|    iterations           | 480            |
|    time_elapsed         | 2002           |
|    total_timesteps      | 983040         |
| train/                  |                |
|    approx_kl            | 1.00408215e-08 |
|    clip_fraction        | 0              |
|    clip_range           | 0.2            |
|    entropy_loss         | -5.68          |
|    explained_variance   | 0              |
|    learning_rate        | 0.0001         |
|    loss                 | 2.17e+12       |
|    n_updates            | 4790           |
|    policy_gradient_loss | -9.71e-07      |
|    reward      

-------------------------------------------
| time/                   |               |
|    fps                  | 491           |
|    iterations           | 486           |
|    time_elapsed         | 2026          |
|    total_timesteps      | 995328        |
| train/                  |               |
|    approx_kl            | 1.0273652e-08 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.68         |
|    explained_variance   | 5.96e-08      |
|    learning_rate        | 0.0001        |
|    loss                 | 1.97e+12      |
|    n_updates            | 4850          |
|    policy_gradient_loss | -1.65e-06     |
|    reward               | 113130.21     |
|    std                  | 1             |
|    value_loss           | 3.88e+12      |
-------------------------------------------
begin_total_asset:100000
end_total_asset:157853.7280768776
Sharpe:  0.7196904027783227
begin_total_asset:100000
end_tota

In [25]:
trade = data_split(df, '2022-01-01', '2022-01-09') # '2021-12-20', '2023-01-01'
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [26]:
import torch
%matplotlib inline
import plotly.express as px

In [27]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import objective_functions
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [39]:
from pyfolio import timeseries

df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model_ppo, environment=e_trade_gym)
time_ind = pd.Series(df_daily_return_ppo.date)
ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

perf_func = timeseries.perf_stats 

perf_stats_all_ppo = perf_func(returns=DRL_strat_ppo, 
                               factor_returns=DRL_strat_ppo, 
                               positions=None, transactions=None, turnover_denom="AGB")

begin_total_asset:100000
end_total_asset:104816.57097641527
Sharpe:  17.621255298064995
hit end!
begin_total_asset:100000
end_total_asset:103857.8987910049
Sharpe:  14.683056946413192
hit end!
begin_total_asset:100000
end_total_asset:103802.51261331959
Sharpe:  15.690859111164913
hit end!
begin_total_asset:100000
end_total_asset:104538.84880694697
Sharpe:  19.298962222094822
hit end!
begin_total_asset:100000
end_total_asset:103648.74320888754
Sharpe:  17.797315989519284
hit end!
begin_total_asset:100000
end_total_asset:106073.68777071327
Sharpe:  23.58452376563577
hit end!
begin_total_asset:100000
end_total_asset:105689.83159249542
Sharpe:  22.98023863621066
hit end!
begin_total_asset:100000
end_total_asset:103840.9803000659
Sharpe:  13.413210819828803
hit end!
begin_total_asset:100000
end_total_asset:104593.19311717042
Sharpe:  18.212104249784414
hit end!
begin_total_asset:100000
end_total_asset:104096.4428924973
Sharpe:  14.48664870429435
hit end!
begin_total_asset:100000
end_total_a